<a href="https://colab.research.google.com/github/dragonsan17/crime_det/blob/master/model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import csv

2.2.0


In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
file1_path = "/content/gdrive/My Drive/f.csv"
file2_path = "/content/gdrive/My Drive/out.csv"
with open(file1_path) as csvfile:
  readCSV = csv.reader(csvfile, delimiter=',')
  file1 = list(readCSV)
with open(file2_path) as csvfile:
  readCSV = csv.reader(csvfile, delimiter=',')
  file2 = list(readCSV)
print(file1[0:2])
print(file2[0:2])

[['murder', '1', "['shock', 'journalist', 'safe', 'pakistan', 'murder', 'journalist', 'req']", '@abbasz55 @jamilnagri Shocking, journalists are no where safe in Pakistan, murder of journalist #AliSherRajjpur req… https://t.co/XAdshoUERH'], ['murder', '1', "['brutal', 'murder', 'girl', 'report', 'bahu', 'plaza', 'area', 'more', 'detail', 'await']", 'Brutal Murder of a girl has been reported in Bahu Plaza area. More details awaited https://t.co/V499wUiYYe']]
[['murder', '1', "['murder']", '@Ndlotus1 @t_d_h_nair @ameytirodkar Murdered'], ['murder', '1', "['look', 'paid', 'analyst', 'wz', 'tweet', 'zainab', 'murder', 'case', 'hous', 'km', 'away', 'rm', 'Fa']", 'Look at this 👇👇how these paid analyst wz tweeting during Zainab murder case, now his house is just 3km away 4rm  Fa… https://t.co/WMVsurZHV9']]


In [7]:
# prepare Data by santosh
def data_prep(data):
  token_list = []
  label_list = []

  for crime,num,s_token,tweet in data:
    token_list.append(s_token[1:-1].split(', '))
    label_list.append(max(int(num),0))

  for i in range(len(token_list)):
    for j in range(len(token_list[i])):
      token_list[i][j] = token_list[i][j][1:-1]
  # print(len(token_list))
  tokens = {}
  crime_info = {}
  # print(len(data))
  for i in range(len(data)):
    tokens[data[i][3]] = token_list[i]
    crime_info[data[i][3]] = []
  for i in range(len(data)):
    crime_info[data[i][3]].append(max(int(data[i][1]),0))

  x = []
  y = []

  for tweet in tokens:
    x.append(tokens[tweet])
    label = 0
    for num in crime_info[tweet]:
      label = max(num,0)
    y.append(label)
  return x,y

x_train,y_train = data_prep(file1)
x_valid,y_valid = data_prep(file2)

training_labels_final = np.array(y_train).astype(int)
testing_labels_final = np.array(y_valid).astype(int)

# print(x_train[0:2])
# print(x_valid[0:2])
# print(y_train[0:2])
# print(y_valid[0:2])

[['shock', 'journalist', 'safe', 'pakistan', 'murder', 'journalist', 'req'], ['brutal', 'murder', 'girl', 'report', 'bahu', 'plaza', 'area', 'more', 'detail', 'await']]
[['murder'], ['look', 'paid', 'analyst', 'wz', 'tweet', 'zainab', 'murder', 'case', 'hous', 'km', 'away', 'rm', 'Fa']]
[1, 1]
[1, 1]


In [0]:
vocab_size = 10000
embedding_dim = 64
max_length = 100
trunc_type='post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(x_train)
x_train_padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(x_valid)
x_valid_padded = pad_sequences(testing_sequences,maxlen=max_length)

# print(x_train_padded[0:2])
# print(x_valid_padded[0:2])

In [62]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 100, 64)           640000    
_________________________________________________________________
flatten_17 (Flatten)         (None, 6400)              0         
_________________________________________________________________
dense_33 (Dense)             (None, 6)                 38406     
_________________________________________________________________
dense_34 (Dense)             (None, 1)                 7         
Total params: 678,413
Trainable params: 678,413
Non-trainable params: 0
_________________________________________________________________


In [63]:
num_epochs = 10
model.fit(x_train_padded, training_labels_final, epochs=num_epochs, validation_data=(x_valid_padded, testing_labels_final))

Epoch 1/10
13/13 [==============================] - 0s 26ms/step - loss: 0.6906 - accuracy: 0.5422 - val_loss: 0.6738 - val_accuracy: 0.6031
Epoch 2/10
13/13 [==============================] - 0s 17ms/step - loss: 0.6790 - accuracy: 0.5652 - val_loss: 0.6764 - val_accuracy: 0.6031
Epoch 3/10
13/13 [==============================] - 0s 17ms/step - loss: 0.6611 - accuracy: 0.7238 - val_loss: 0.6641 - val_accuracy: 0.6031
Epoch 4/10
13/13 [==============================] - 0s 18ms/step - loss: 0.6050 - accuracy: 0.6777 - val_loss: 0.6593 - val_accuracy: 0.6186
Epoch 5/10
13/13 [==============================] - 0s 17ms/step - loss: 0.5236 - accuracy: 0.9386 - val_loss: 0.6383 - val_accuracy: 0.6340
Epoch 6/10
13/13 [==============================] - 0s 16ms/step - loss: 0.4138 - accuracy: 0.9540 - val_loss: 0.6168 - val_accuracy: 0.6675
Epoch 7/10
13/13 [==============================] - 0s 17ms/step - loss: 0.2982 - accuracy: 0.9591 - val_loss: 0.5948 - val_accuracy: 0.7036
Epoch 8/10
13